In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [2]:
X_train, X_test, y_train, y_test = (
    pd.read_csv('data/X_train_extended.csv'),
    pd.read_csv('data/X_test_extended.csv'),
    pd.read_csv('data/y_train.csv'),
    pd.read_csv('data/y_test.csv')
)

In [3]:
X_train, X_test = (
    X_train.drop(columns=['Type_L', 'Type_M']),
    X_test.drop(columns=['Type_L', 'Type_M'])
)

y_train, y_test = y_train['Failure type'], y_test['Failure type']

In [4]:
y_train.values.reshape(-1, 1).shape

(8000, 1)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.utils import class_weight
import numpy as np

2024-07-26 14:31:09.625166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 14:31:09.625774: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 14:31:09.628626: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 14:31:09.636438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 14:31:09.650062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [6]:
# Define the model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(7,)),  # 7 input features
    layers.Dense(32, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 classes for the output
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['recall', 'f1_score'])

/home/carroyo/personal_python_projects/predictive-maintenance-milling/tmp/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1722025871.352170  401422 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-26 14:31:11.352499: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
isinstance(model, tf.keras.models.Model)

True

# Experimentos

In [8]:
from mltools import results
from mltools import models as models_tools

In [9]:
nn_wrapper = models_tools.NeuralNetworkWrapper(model=model, name='neural_network')

In [10]:
nn_wrapper.fit(X_train, y_train, sampling_strategy=None)

Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (8000,)
Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (8000,)
Type of y_train_encoded: <class 'numpy.ndarray'>
Shape of y_train_encoded: (8000, 7)
Type of X_train: <class 'pandas.core.frame.DataFrame'>
Shape of X_train: (8000, 7)
Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.0223 - loss: 2.1110 - recall: 3.8698e-04 - val_f1_score: 0.1029 - val_loss: 1.8190 - val_recall: 0.0044
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.1254 - loss: 1.3209 - recall: 0.0173 - val_f1_score: 0.1587 - val_loss: 1.4911 - val_recall: 0.1381
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.1820 - loss: 1.2337 - recall: 0.1832 - val_f1_score: 0.1816 - val_loss: 1.3453 - val_recall: 0.2450
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.2062 - loss: 0.9352 - recall: 0.2479 - val_f1_score: 0.1907 - val_loss: 1.2671 - val_recall: 0.3262
Epoch 

In [11]:
experiment = results.ExperimentAutomation([nn_wrapper])

In [12]:
y_train.shape

(8000,)

In [15]:
experiment.run_experiment(X_train, X_test, y_train, y_test)

Shape of y_train: (8000,) in experiments
Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (8000,)
Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (8000,)
Type of y_train_encoded: <class 'numpy.ndarray'>
Shape of y_train_encoded: (8000, 7)
Type of X_train: <class 'pandas.core.frame.DataFrame'>
Shape of X_train: (8000, 7)
Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3432 - loss: 0.4472 - recall: 0.6014 - val_f1_score: 0.3203 - val_loss: 0.7162 - val_recall: 0.6625
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3307 - loss: 0.4662 - recall: 0.6449 - val_f1_score: 0.3194 - val_loss: 0.9226 - val_recall: 0.5919
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3521 - loss: 0.3896 - recall: 0.6275 - val_f1_score: 0.3039 - val_loss: 1.0045 - val_recall: 0.5387
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - f1_score: 0.3302 - loss: 0.4191 - recall: 0.5855 - val_f1_score: 0.2947 - val_los

In [16]:
display(experiment.results.results_df.shape)

(40, 5)

In [18]:
display(experiment.results.results_df.round(2))

,precision,recall,f1-score,support,model
0,1.00,0.66,0.80,1930.00,neural_network
1,0.06,0.89,0.11,9.00,neural_network
2,0.28,0.90,0.43,21.00,neural_network
3,0.25,1.00,0.40,16.00,neural_network
4,0.22,0.81,0.34,16.00,neural_network
5,0.00,0.00,0.00,3.00,neural_network
6,0.42,1.00,0.59,5.00,neural_network
accuracy,0.67,0.67,0.67,0.67,neural_network
macro avg,0.32,0.75,0.38,2000.00,neural_network
weighted avg,0.97,0.67,0.78,2000.00,neural_network
